In [17]:
import cv2
import numpy as np
import os
import copy
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from sklearn.cluster import MeanShift
from random import randint
#from detector import Detectors
from scipy.spatial import distance as dist
import collections
from collections import OrderedDict
from random import randint
from math import sqrt

In [18]:
class Cell:
    def __init__(self, param):
        self.frame = param[0]
        self.linking_Id = -1
        self.center = param[1]

        self.long_axis = param[2][1]
        self.short_axis = param[2][0]
        self.ratio = self.ratio_axis()
        self.status = self.find_status(self.ratio)
    def update_id(self,ids):
        self.linking_Id=ids
    def ratio_axis(self):
        return self.short_axis/self.long_axis
    def find_status(self,ratio):
        if ratio<0.16:
            return 1
        else:
            return 0

In [19]:
def gray_img(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [20]:
def normalize(image):
    img = image.copy().astype(np.float32)
    img -= np.mean(img)
    img /= np.linalg.norm(img)
    img = np.clip(img, 0, 255)
    img *= (1./float(img.max()))
    return (img*255).astype(np.uint8)

In [21]:
def read_img(path):
    sequence_tif = {}
    gary_seq_tif = {}
    for root, dirs, files, in os.walk(path):
        dirs.sort()
        for dir in dirs:
            sequence_tif[dir] = []
            gary_seq_tif[dir] = []
            for _, _, files_1 in os.walk(os.path.join(path, dir)):
                files_1.sort()
                for file in files_1:
                    new_dir_path = os.path.join(path, dir)
                    temp_img = cv2.imread(os.path.join(new_dir_path, file))
                    gray_imgs = gray_img(temp_img.copy())
                    sequence_tif[dir].append(temp_img)
                    gary_seq_tif[dir].append(gray_imgs)

    return sequence_tif,gary_seq_tif

In [22]:
def dist(a, b):
    return sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [7]:
def dict_cells(sequence):
    #cell=Features()
    dict_cell={}
    list_cell=[]
    #frame_cell={}
    #contours1=[] 
    center_sl=[]
    for k in range(len(sequence)):
        #cell=Features()
        #former_list_centers=[]
        frame_cell={}
        center_elli=[]
        frame=sequence[k]
        f=copy.copy(frame)
        frame_cell['frame']=k
        frame_cell['draw_img']=np.asarray(f)
        frame_cell['feature']=[]
        imgO=normalize(f)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1=[] 
        division_cells=[]
        #center_sl=[]
        num_divide=0
        for i in contours:
            if cv2.contourArea(i)>25:  
                contours1.append(i)
        for m,n in zip(contours1,range(len(contours1))):
            M = cv2.moments(m)
            cX=int(M["m10"]/M["m00"])
            cY=int(M["m01"]/M["m00"])
            ellipse=cv2.fitEllipse(m)
            s_l=ellipse[1][0]/ellipse[1][1]
                    #ellipse[1][0]
                    #center_sl.append([k,(cX,cY),(ellipse[1][0],ellipse[1][1])])
            if s_l<0.16:
                division_cells.append(m)
            frame_cell['feature'].append([k,(cX,cY),(ellipse[1][0],ellipse[1][1])])
        division_num=len(division_cells)
        count_num=len(contours1)-1
        #division_num=len(division_cells)
        frame_cell['count']=count_num
        cv2.drawContours(f,contours1,-1,(0,255,0),1)
        cv2.drawContours(f,division_cells,-1,(255,255,0),2)
        cv2.putText(f, 'number of cells: ' + str(count_num), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 0, 0), 2, cv2.LINE_AA, False)
        cv2.putText(f, 'number of divided cells: ' + str(division_num), (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 255, 0), 2, cv2.LINE_AA, False)
        dict_cell[k]=frame_cell
    return dict_cell

In [15]:
def record_dict(path2):
    original_img,grays=read_img(path2)

    return dict_cells(original_img['Sequence 1'])

In [13]:
path='D:/jupyter/9517_porject/data/'

In [14]:
path2=path+'PhC-C2DL-PSC'

In [16]:
maxD = 600
cellSpaceTime = []

In [17]:
dict_cell=record_dict(path2)

In [27]:
result=[dict_cell[i]['feature'] for i in range(426)]

In [31]:
tracking_cells=[i for i in range(dict_cell[0]['count'])]

In [39]:
num_frame=426

In [40]:
initial_cells = []
for idx in tracking_cells:
    cell = Cell(dict_cell[0]['feature'][idx])
    cell.update_id(idx)
    initial_cells.append(cell)
cellSpaceTime.append(initial_cells)

In [41]:
for frame in range(1, num_frame):
    cells = [Cell(e) for e in dict_cell[frame]['feature']]
    cellSpaceTime.append(cells)

In [42]:
for frame in range(num_frame - 1):
    f_img = dict_cell[frame]['draw_img']
    #divide_num=0
    for cur in cellSpaceTime[frame]:
        nex_0 = cellSpaceTime[frame + 1][0]
        d_min, match = dist(cur.center, nex_0.center), nex_0
        for nex in cellSpaceTime[frame + 1]:
            d = dist(cur.center, nex.center)
            if d < d_min:
                d_min, match = d, nex
        if d_min < maxD and not (cur.status == 1):
            if cur.linking_Id == -1:
                cur.update_id(tracking_cells[-1] + 1)
                tracking_cells.append(cur.linking_Id)
            match.update_id(cur.linking_Id)
        #if d_min < maxD and cur.status == 1:
            #divide_num+=1
    #cv2.putText(f_img, 'number of divided cells: ' + str(divide_num), (10, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 255, 0), 2, cv2.LINE_AA, False)

In [43]:
trajectories = {}
for time in cellSpaceTime:
    for cell in time:
        if cell.linking_Id == -1:
            continue
        if cell.linking_Id not in trajectories:
            trajectories[cell.linking_Id] = [(cell.frame, cell.center)]
        else:
            trajectories[cell.linking_Id].append((cell.frame, cell.center))

In [44]:
for frame in range(num_frame - 1):
    trajectories = {}
    for time in cellSpaceTime:
        for cell in time:
            if cell.linking_Id == -1:
                continue
            if cell.linking_Id not in trajectories:
                trajectories[cell.linking_Id] = [(cell.frame, cell.center)]
            else:
                trajectories[cell.linking_Id].append((cell.frame, cell.center))

In [67]:
frameNo = randint(0, 425)
print('frame:', frameNo)
paths = {}
for idx in trajectories:
    for i, item in enumerate(trajectories[idx]):
        if item[0] == frameNo:
            path = list(zip(*trajectories[idx][:i + 1]))[1]
            paths[idx] = path
            break

colors = tracking_cells
print('num of colors:', len(colors))
pigment = {i: (randint(0, 255), randint(0, 255), randint(0, 255)) for i in colors}

frame: 387
num of colors: 6586


In [45]:
drawing=[]
for fnum in range(10):
    drawing.append(dict_cell[fnum]['draw_img'])

In [ ]:
u=[]
for t in range(10):
    paths = {}
    for idx in trajectories:
        for i, item in enumerate(trajectories[idx]):
            if item[0] == t:
                path = list(zip(*trajectories[idx][:i + 1]))[1]
                paths[idx] = path
    u.append(paths)

print(u)  

In [51]:
path_data=np.array(u)
final_draw=np.array(drawing)
print(path_data.shape)
print(final_draw.shape)

(10,)
(10, 576, 720, 3)


In [52]:
from drawCalculate import draw_and_calculate
draws, outputs = draw_and_calculate(path_data, final_draw)

In [ ]:
draw = dict_cell[frameNo]['draw_img']
for n, path in paths.items():
    print('n:', n)
    print('path:', path)
    pts = [np.array(path, dtype=np.int32)]
    draw = cv2.polylines(draw, pts, isClosed=False,color=pigment[n], thickness=3) 
plt.imsave('trackingPhc_'+'86'+'.jpg',draw)

In [30]:
for frameNo in range(10):
    draw = dict_cell[frameNo]['draw_img']
    for n, path in paths.items():
        #print('n:', n)
        #print('path:', path)
        pts = [np.array(path, dtype=np.int32)]
        draw = cv2.polylines(draw, pts, isClosed=False,color=pigment[n], thickness=3)

    plt.imsave('trackingPhc_overall' + str(frameNo) + '.jpg', draw)